In [1]:
import numpy as np

x = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
y = np.array([1, 3, 2, 5, 7, 8, 8, 9, 10, 12])

def analytical_linear_regression(x, y):
    n = len(x)
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    xy_mean = np.mean(x * y)
    x_squared_mean = np.mean(x**2)

    slope = (xy_mean - x_mean * y_mean) / (x_squared_mean - x_mean**2)
    intercept = y_mean - slope * x_mean

    y_pred = slope * x + intercept

    sse = np.sum((y - y_pred)**2)

    ssr = np.sum((y_pred - y_mean)**2)
    sst = np.sum((y - y_mean)**2)
    r_squared = ssr / sst

    return slope, intercept, sse, r_squared

def gradient_descent_linear_regression(x, y, learning_rate=0.01, epochs=1000, tol=1e-6):
    n = len(x)
    slope = 0
    intercept = 0
    prev_slope = 1
    prev_intercept = 1
    iter_count = 0

    while iter_count < epochs and abs(prev_slope - slope) > tol and abs(prev_intercept - intercept) > tol:
        prev_slope = slope
        prev_intercept = intercept
        y_pred = slope * x + intercept
        slope -= (2 / n) * learning_rate * np.sum((y_pred - y) * x)
        intercept -= (2 / n) * learning_rate * np.sum(y_pred - y)
        iter_count += 1

    sse = np.sum((y - (slope * x + intercept))**2)
    y_mean = np.mean(y)
    ssr = np.sum((slope * x + intercept - y_mean)**2)
    sst = np.sum((y - y_mean)**2)
    r_squared = ssr / sst

    return slope, intercept, sse, r_squared

slope_analytical, intercept_analytical, sse_analytical, r_squared_analytical = analytical_linear_regression(x, y)
print("Analytical Solution:")
print("Slope:", slope_analytical)
print("Intercept:", intercept_analytical)
print("Sum of Squared Error (SSE):", sse_analytical)
print("R-squared:", r_squared_analytical)
slope_gradient, intercept_gradient, sse_gradient, r_squared_gradient = gradient_descent_linear_regression(x, y)
print("\nGradient Descent Solution:")
print("Slope:", slope_gradient)
print("Intercept:", intercept_gradient)
print("Sum of Squared Error (SSE):", sse_gradient)
print("R-squared:", r_squared_gradient)

Analytical Solution:
Slope: 1.1696969696969695
Intercept: 1.2363636363636372
Sum of Squared Error (SSE): 5.624242424242422
R-squared: 0.9525380386139874

Gradient Descent Solution:
Slope: 1.170263693076768
Intercept: 1.2328099487610318
Sum of Squared Error (SSE): 5.624278989977716
R-squared: 0.9534613650109194


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

housing_data = pd.read_csv("housing.csv")

selected_attribute = 'median_income'
X = housing_data[selected_attribute].values.reshape(-1, 1)
y = housing_data['median_house_value'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_with_intercept = np.c_[np.ones(X_train.shape[0]), X_train]
X_test_with_intercept = np.c_[np.ones(X_test.shape[0]), X_test]

theta_analytic = np.linalg.inv(X_train_with_intercept.T.dot(X_train_with_intercept)).dot(X_train_with_intercept.T).dot(y_train)
print("Coefficients using Analytic Formulation:", theta_analytic)

def full_batch_gradient_descent(X, y, learning_rate, num_iterations):
    theta = np.zeros(X.shape[1])
    for _ in range(num_iterations):
        y_pred = X.dot(theta)
        theta -= (1/len(y)) * learning_rate * X.T.dot(y_pred - y)
    return theta

learning_rate = 0.01
num_iterations = 1000
theta_full_batch = full_batch_gradient_descent(X_train_with_intercept, y_train, learning_rate, num_iterations)
print("Coefficients using Full-batch Gradient Descent:", theta_full_batch)

def stochastic_gradient_descent(X, y, learning_rate, num_iterations):
    theta = np.zeros(X.shape[1])
    for _ in range(num_iterations):
        for i in range(len(y)):
            rand_index = np.random.randint(0, len(y))
            xi = X[rand_index]
            yi = y[rand_index]
            y_pred = np.dot(xi, theta)
            theta -= learning_rate * xi * (y_pred - yi)
    return theta

theta_stochastic = stochastic_gradient_descent(X_train_with_intercept, y_train, learning_rate, num_iterations)
print("Coefficients using Stochastic Gradient Descent:", theta_stochastic)
y_pred_analytic = X_test_with_intercept.dot(theta_analytic)
y_pred_full_batch = X_test_with_intercept.dot(theta_full_batch)
y_pred_stochastic = X_test_with_intercept.dot(theta_stochastic)

SSE_analytic = np.sum((y_test - y_pred_analytic) ** 2)
SSE_full_batch = np.sum((y_test - y_pred_full_batch) ** 2)
SSE_stochastic = np.sum((y_test - y_pred_stochastic) ** 2)

mean_y_test = np.mean(y_test)
SST = np.sum((y_test - mean_y_test) ** 2)

R_squared_analytic = 1 - (SSE_analytic / SST)
R_squared_full_batch = 1 - (SSE_full_batch / SST)
R_squared_stochastic = 1 - (SSE_stochastic / SST)

print("SSE and R-squared value:")
print("Analytic Formulation: SSE =", SSE_analytic, ", R-squared =", R_squared_analytic)
print("Full-batch Gradient Descent: SSE =", SSE_full_batch, ", R-squared =", R_squared_full_batch)
print("Stochastic Gradient Descent: SSE =", SSE_stochastic, ", R-squared =", R_squared_stochastic)

Coefficients using Analytic Formulation: [44459.72916908 41933.84939381]
Coefficients using Full-batch Gradient Descent: [39148.47787113 43047.96802282]
Coefficients using Stochastic Gradient Descent: [39699.61079795 43224.33606282]
SSE and R-squared value:
Analytic Formulation: SSE = 29272299281848.184 , R-squared = 0.4588591890384667
Full-batch Gradient Descent: SSE = 29321631561932.883 , R-squared = 0.4579472104543938
Stochastic Gradient Descent: SSE = 29322871000039.15 , R-squared = 0.4579242976474598
